Set `linkages_path` to the location of the csv containg dataset-publication linkages and read in csv

In [2]:
import pandas as pd

Update column names in lines 3/4

In [40]:
linkages_path =  '/Users/sophierand/RichContextMetadata/metadata/20190717_usda_snap/SNAP_DATA_DIMENSIONS_SEARCH_DEMO.csv'
linkages_csv = pd.read_csv(linkages_path)
linkages_csv = linkages_csv.loc[pd.notnull(linkages_csv.dataset)].drop_duplicates()
linkages_csv = linkages_csv.loc[pd.notnull(linkages_csv.title)].drop_duplicates()

In [4]:
import unicodedata

In [5]:
def scrub_unicode (text):
    """
    try to handle the unicode edge cases encountered in source text,
    as best as possible
    """
    x = " ".join(map(lambda s: s.strip(), text.split("\n"))).strip()

    x = x.replace('“', '"').replace('”', '"')
    x = x.replace("‘", "'").replace("’", "'").replace("`", "'")
    x = x.replace("`` ", '"').replace("''", '"')
    x = x.replace('…', '...').replace("\\u2026", "...")
    x = x.replace("\\u00ae", "").replace("\\u2122", "")
    x = x.replace("\\u00a0", " ").replace("\\u2022", "*").replace("\\u00b7", "*")
    x = x.replace("\\u2018", "'").replace("\\u2019", "'").replace("\\u201a", "'")
    x = x.replace("\\u201c", '"').replace("\\u201d", '"')

    x = x.replace("\\u20ac", "€")
    x = x.replace("\\u2212", " - ") # minus sign

    x = x.replace("\\u00e9", "é")
    x = x.replace("\\u017c", "ż").replace("\\u015b", "ś").replace("\\u0142", "ł")    
    x = x.replace("\\u0105", "ą").replace("\\u0119", "ę").replace("\\u017a", "ź").replace("\\u00f3", "ó")

    x = x.replace("\\u2014", " - ").replace('–', '-').replace('—', ' - ')
    x = x.replace("\\u2013", " - ").replace("\\u00ad", " - ")

    x = str(unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8"))

    # some content returns text in bytes rather than as a str ?
    try:
        assert type(x).__name__ == "str"
    except AssertionError:
        print("not a string?", type(x), x)

    return x

In [31]:
pub_metadata_fields = ['title','dataset']
original_metadata_cols = list(set(linkages_csv.columns.values.tolist()) - set(pub_metadata_fields))

In [32]:
a = linkages_csv.head()

In [ ]:
def create_pub_dict(linkages_dataframe):

In [44]:
def create_pub_dict(linkages_dataframe):
    pub_dict_list = []
    for i, r in linkages_dataframe.iterrows():
        r['title'] = scrub_unicode(r['title'])
        required_metadata = r[pub_metadata_fields].to_dict()
        original_metadata = r[original_metadata_cols].to_dict()
        pub_dict = required_metadata
        pub_dict.update({'original_metadata':original_metadata})
        pub_dict_list.append(pub_dict)
    return pub_dict_list

In [45]:
    pub_dict


{'title': "Farmers' Markets and the Local Food Environment: Identifying Perceived Accessibility Barriers for SNAP Consumers Receiving Temporary Assistance for Needy Families (TANF) in an Urban Oklahoma Community",
 'dataset': 'dataset-030',
 'original_metadata': {'icrp_cso_1': nan,
  'rcdc_categories': 'Clinical Research',
  'state_of_research_organization': 'Oklahoma; Oklahoma',
  'publication_id': 'pub.1050299597',
  'uids_of_supporting_grants': 'grant.2438431; grant.2437893',
  'mesh_terms': 'Adolescent; Adult; Attitude; Female; Food Assistance; Food Supply; Fruit; Humans; Male; Middle Aged; Oklahoma; Poverty; Vegetables; Young Adult',
  'hrcs_hc_5': nan,
  'authors_affiliations': 'Wetherill, Marianna S. (University of Oklahoma Health Sciences Center); Gray, Karen A. (University of Oklahoma)',
  'hrcs_rac_5': nan,
  'icrp_cso_categories': nan,
  'grid_ids': 'grid.266900.b; grid.266902.9',
  'hrcs_rac_categories': nan,
  'recent_citations': 15,
  'dimensions_url': 'https://app.dimens

In [38]:
# linkages_csv.head()

In [6]:
# linkages_csv = linkages_csv[pub_metadata_fields]
# linkages_csv['title'] = linkages_csv['title'].apply(scrub_unicode)

#### Generate list of dicts of metadata

Read in `datasets.json`. Update `datasets_path` to your local.

In [46]:
import json

In [47]:
datasets_path = '/Users/sophierand/RCDatasets/datasets.json'

with open(datasets_path) as json_file:
    datasets = json.load(json_file)

Create list of dictionaries of publication metadata. `format_metadata` iterrates through `linkages_csv` dataframe, splits the `dataset_id` field (for when multiple datasets are listed); throws an error if the dataset doesn't exist and needs to be added to `datasets.json`.

NOTE: Update any column names as needed.

In [52]:
def create_pub_dict(linkages_dataframe,datasets):
    pub_dict_list = []
    for i, r in linkages_dataframe.iterrows():
        r['title'] = scrub_unicode(r['title'])
        ds_id_list = [d.strip() for d in r['dataset'].split(",")]
        for ds in ds_id_list:
            check_ds = [b for b in datasets if b['id'] == ds]
            if len(check_ds) == 0:
                print('dataset {} isnt listed in datasets.json. Please add to file'.format(ds))
        required_metadata = r[pub_metadata_fields].to_dict()
        required_metadata.update({'datasets':ds_id_list})
        original_metadata = r[original_metadata_cols].to_dict()
        pub_dict = required_metadata
        pub_dict.update({'original_metadata':original_metadata})
        pub_dict_list.append(pub_dict)
    return pub_dict_list

In [53]:
create_pub_dict(a,datasets)

[{'title': "SNAP-Based Incentive Programs at Farmers' Markets: Adaptation Considerations for Temporary Assistance for Needy Families (TANF) Recipients",
  'dataset': 'dataset-026',
  'datasets': ['dataset-026'],
  'original_metadata': {'icrp_cso_1': nan,
   'rcdc_categories': 'Clinical Trials and Supportive Activities; Clinical Research',
   'state_of_research_organization': 'Oklahoma; Oklahoma',
   'publication_id': 'pub.1091208742',
   'uids_of_supporting_grants': 'grant.2438431; grant.2437893',
   'mesh_terms': 'Adult; Female; Food Assistance; Food Supply; Health Behavior; Health Promotion; Humans; Male; Motivation; Young Adult',
   'hrcs_hc_5': nan,
   'authors_affiliations': 'Wetherill, Marianna S. (University of Oklahoma Health Sciences Center); Williams, Mary B. (University of Oklahoma Health Sciences Center); Gray, Karen A. (University of Oklahoma)',
   'hrcs_rac_5': nan,
   'icrp_cso_categories': nan,
   'grid_ids': 'grid.266902.9; grid.266900.b',
   'hrcs_rac_categories': nan

In [9]:
def format_linkages(linkages_csv,datasets):
    linkage_list = []
    for i,r in linkages_csv.iterrows():
        ds_id_list = [d.strip() for d in r['dataset_id'].split(",")]
        pub_dict = {'title':r['title'],'url':r['pub_url'],'datasets':ds_id_list}
        for ds in ds_id_list:
            check_ds = [b for b in datasets if b['id'] == ds]
            if len(check_ds) == 0:
                print('dataset {} isnt listed in datasets.json. Please add to file'.format(ds))
        linkage_list.append(pub_dict)
    return linkage_list

Generate publication metadata and export to json

In [10]:
linkage_list = format_linkages(linkages_csv,datasets)

Update `pub_path` to be: 
`<name_of_subfolder>_publications.json`

In [18]:
import os
pub_path = '20190920_excel_usda_publications.json'
json_pub_path = os.path.join('/Users/sophierand/RCPublications/partitions/',pub_path)

In [19]:
with open(json_pub_path, 'w') as outfile:
    json.dump(linkage_list, outfile, indent=2)